In [1]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import time

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
mnist = input_data.read_data_sets('MNIST/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print(mnist.train.images.shape)

(55000, 784)


In [4]:
n_hidden_1 = 256
n_hidden_2 = 128
n_classes = 10
n_input = 784

x = tf.placeholder('float', [None, n_input])
y = tf.placeholder('float', [None, n_classes])

stddev = 0.1 
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev)),

}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes])),

}



In [5]:
def multilayer_perceptron(x, weights, biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['w1']), biases['b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['w2']), biases['b2']))
    return (tf.matmul(layer_2, weights['out']) + biases['out'])

In [6]:
pred = multilayer_perceptron(x, weights, biases)

# loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optm = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost)
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(corr, 'float'))

# Initializer
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [8]:
start_time = time.time()
train_epochs = 100
batch_size = 100
display_step = 4
sess = tf.Session()
sess.run(init)

for epoch in range(train_epochs):
    avg_cost = 0
    num_batch = int(mnist.train.num_examples/batch_size)
    for i in range(num_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost/num_batch
    if (epoch+1) % display_step ==0:
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        feeds = {x: mnist.test.images, y: mnist.test.labels}
        test_acc = sess.run(accr, feed_dict=feeds)
        print('Epoch:%4d  train_acc:%.8f    test_acc:%.8f    cost:%.8f'
             %(epoch, train_acc, test_acc, avg_cost))
print('Finished')   
end_time = time.time()
print('The time is:', end_time-start_time)

Epoch:   3  train_acc:0.20000000    test_acc:0.18300000    cost:2.26819175
Epoch:   7  train_acc:0.40000001    test_acc:0.36059999    cost:2.23078225
Epoch:  11  train_acc:0.50000000    test_acc:0.47700000    cost:2.18954163
Epoch:  15  train_acc:0.56999999    test_acc:0.54960001    cost:2.14207584
Epoch:  19  train_acc:0.62000000    test_acc:0.59939998    cost:2.08604145
Epoch:  23  train_acc:0.61000001    test_acc:0.62210000    cost:2.01939000
Epoch:  27  train_acc:0.66000003    test_acc:0.63300002    cost:1.94066209
Epoch:  31  train_acc:0.63999999    test_acc:0.65750003    cost:1.84936473
Epoch:  35  train_acc:0.72000003    test_acc:0.66979998    cost:1.74705981
Epoch:  39  train_acc:0.64999998    test_acc:0.68099999    cost:1.63743068
Epoch:  43  train_acc:0.72000003    test_acc:0.70020002    cost:1.52589738
Epoch:  47  train_acc:0.68000001    test_acc:0.71120000    cost:1.41808820
Epoch:  51  train_acc:0.67000002    test_acc:0.72689998    cost:1.31813020
Epoch:  55  train_acc:0.7

In [7]:
start_time = time.time()
train_epochs = 100
batch_size = 100
display_step = 4
sess = tf.Session()
sess.run(init)

for epoch in range(train_epochs):
    avg_cost = 0
    num_batch = int(mnist.train.num_examples/batch_size)
    for i in range(num_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost/num_batch
    if (epoch+1) % display_step ==0:
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        feeds = {x: mnist.test.images, y: mnist.test.labels}
        test_acc = sess.run(accr, feed_dict=feeds)
        print('Epoch:%4d  train_acc:%.8f    test_acc:%.8f    cost:%.8f'
             %(epoch, train_acc, test_acc, avg_cost))
print('Finished')   
end_time = time.time()
print('The time is:', end_time-start_time)

Epoch:   3  train_acc:0.14000000    test_acc:0.21660000    cost:2.26677404
Epoch:   7  train_acc:0.28999999    test_acc:0.35389999    cost:2.22870895
Epoch:  11  train_acc:0.37000000    test_acc:0.44900000    cost:2.18655298
Epoch:  15  train_acc:0.47000000    test_acc:0.49689999    cost:2.13784413
Epoch:  19  train_acc:0.50999999    test_acc:0.55860001    cost:2.08050165
Epoch:  23  train_acc:0.49000001    test_acc:0.59560001    cost:2.01273979
Epoch:  27  train_acc:0.63999999    test_acc:0.61559999    cost:1.93372621
Epoch:  31  train_acc:0.61000001    test_acc:0.64579999    cost:1.84375938
Epoch:  35  train_acc:0.69000000    test_acc:0.66790003    cost:1.74493889
Epoch:  39  train_acc:0.68000001    test_acc:0.69090003    cost:1.64082003
Epoch:  43  train_acc:0.69999999    test_acc:0.71469998    cost:1.53554776
Epoch:  47  train_acc:0.74000001    test_acc:0.73519999    cost:1.43309734
Epoch:  51  train_acc:0.73000002    test_acc:0.74680001    cost:1.33649603
Epoch:  55  train_acc:0.7